# 전처리를 해보자
## - 프로젝트에서 할 것
- 1. 일단 기본값은 전부 칼럼이 존재해야한다.
- 2. 추후 나머지 칼럼들을 삭제해보며 모델 성능을 파악해본다.
- 3. 해당과정을 빠르게 가능하도록 파이프라인을 구축한다.
### - 기본적인 EDA라면 체크사항

1. 데이터 이해 
2. 요약 통계 : dsecribe, boxplot 쓰자
3. 시각화 : 여러개 해보자
4. 상관 관계 탐색 : corrlation 그래프 
5. 이상치 탐색 : 박스 플롯으로 한 번 확인해보자
6. 결측치 처리 : 이 데이터에는 크게 상관없음
7. 가설 수립 : 다 해보고 결론 내보자

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('jung.csv')

In [4]:
df["사고시"] = df["사고일시"].str.split(" ").str.get(3)
df['사고시'] = df['사고시'].str.extract('(\d+)').astype(int)
df['월'] = df['사고일시'].str.extract('(\d+)월').astype(str)
df["도광역시"] = df["시군구"].str.split(" ").str.get(0)

bins = [0, 6, 12, 18, 24]
labels = ['dawn', 'morn', 'aftern', 'night']

df['하루시간구분'] = pd.cut(df['사고시'], bins=bins, labels=labels, right=False)
df['하루시간구분'] = df['하루시간구분'].astype('object')

In [5]:
df['타겟'] = df['가해운전자 상해정도'].map({'경상': 1, '부상신고': 1, '기타불명': 0, '상해없음': 0, '사망': 2, '중상': 2})
df.drop('가해운전자 상해정도', inplace=True, axis=1)

In [6]:
dele = ['사고일시','사고번호','시군구' ,'사고시']
# dele = ['사고일시','사고번호','시군구','사망자수','중상자수','경상자수','부상신고자수']
df.drop(columns=dele, inplace=True)

In [7]:
for column in df.columns:
    unq = df[column].unique()
    print(f"칼럼 {column}의 유니크한 값: {unq}")
    print()

칼럼 요일의 유니크한 값: ['목요일' '수요일' '금요일' '토요일' '월요일' '화요일' '일요일']

칼럼 사고내용의 유니크한 값: ['경상사고' '중상사고' '부상신고사고' '사망사고']

칼럼 사망자수의 유니크한 값: [0 1]

칼럼 중상자수의 유니크한 값: [0 1 2 3]

칼럼 경상자수의 유니크한 값: [ 1  0  2  4  3  5  7  6 10]

칼럼 부상신고자수의 유니크한 값: [0 1 2 3 4 6 5]

칼럼 사고유형의 유니크한 값: ['차대차 - 측면충돌' '차대차 - 정면충돌' '차대차 - 기타' '차대사람 - 차도통행중' '차대차 - 추돌'
 '차대사람 - 길가장자리구역통행중' '차대사람 - 기타' '차대사람 - 보도통행중' '차대사람 - 횡단중' '차대차 - 후진중충돌']

칼럼 법규위반의 유니크한 값: ['교차로운행방법위반' '안전운전불이행' '중앙선침범' '신호위반' '차로위반' '직진우회전진행방해' '안전거리미확보' '기타'
 '보행자보호의무위반' '불법유턴']

칼럼 노면상태의 유니크한 값: ['건조' '젖음/습기' '기타' '적설' '서리/결빙' '침수']

칼럼 기상상태의 유니크한 값: ['맑음' '비' '흐림' '기타' '눈' '안개']

칼럼 도로형태의 유니크한 값: ['교차로 - 교차로안' '단일로 - 기타' '교차로 - 교차로부근' '기타 - 기타' '단일로 - 지하차도(도로)내'
 '교차로 - 교차로횡단보도내' '주차장 - 주차장' '단일로 - 교량' '단일로 - 고가도로위' '단일로 - 터널'
 '미분류 - 미분류']

칼럼 가해운전자 차종의 유니크한 값: ['자전거']

칼럼 가해운전자 성별의 유니크한 값: ['남' '여']

칼럼 가해운전자 연령의 유니크한 값: [70 20 75 76 61 84 59 19 67 11 64 80 79 72 66 18 15 85 63 73 57 81 45 60
 65 39 54 56 22 51 77 53 46 78 25 68 74 52 36 28 47 69 43 55 

In [12]:
# '도광역시' 칼럼에서 '미분류' 값이 몇 번 나타나는지 확인
df = df.drop(df[df['도광역시'] == '미분류'].index)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30372 entries, 0 to 30372
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   요일          30372 non-null  object
 1   사고내용        30372 non-null  object
 2   사망자수        30372 non-null  int64 
 3   중상자수        30372 non-null  int64 
 4   경상자수        30372 non-null  int64 
 5   부상신고자수      30372 non-null  int64 
 6   사고유형        30372 non-null  object
 7   법규위반        30372 non-null  object
 8   노면상태        30372 non-null  object
 9   기상상태        30372 non-null  object
 10  도로형태        30372 non-null  object
 11  가해운전자 차종    30372 non-null  object
 12  가해운전자 성별    30372 non-null  object
 13  가해운전자 연령    30372 non-null  int64 
 14  피해운전자 차종    30372 non-null  object
 15  피해운전자 성별    30372 non-null  object
 16  피해운전자 연령    30372 non-null  int64 
 17  피해운전자 상해정도  30372 non-null  object
 18  월           30372 non-null  object
 19  도광역시        30372 non-null  object
 20  하루시간구분     

In [14]:
# object 타입인 열들을 선택
object_columns = df.select_dtypes(include=['object']).columns
# 해당 열들에 대해서 원핫 인코딩 적용
df = pd.get_dummies(df, columns=object_columns,dtype=int)


In [15]:
df.head()

,사망자수,중상자수,경상자수,부상신고자수,가해운전자 연령,피해운전자 연령,타겟,요일_금요일,요일_목요일,요일_수요일,...,도광역시_인천광역시,도광역시_전라남도,도광역시_전라북도,도광역시_제주특별자치도,도광역시_충청남도,도광역시_충청북도,하루시간구분_aftern,하루시간구분_dawn,하루시간구분_morn,하루시간구분_night
0,0,0,1,0,70,50,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,20,42,2,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,75,81,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,76,75,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,1,0,61,29,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
df.to_csv('prepro_jung.csv', index=False)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30372 entries, 0 to 30372
Columns: 117 entries, 사망자수 to 하루시간구분_night
dtypes: int32(110), int64(7)
memory usage: 14.6 MB
